### Webscraping

**OBJECTIVES**

- Use `BeautifulSoup` to parse HTML 
- Scape websites and structure their data in `DataFrame`
- Build models using text as input
- Use `CountVectorizer` to create numeric representation of text
- Use `selenium` library to interact with web pages

In [ ]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

### HTML

Below is some basic HTML.  It is always housed in tags, and we will use these tags to locate elements of a webpage that we want to extract.

In [ ]:
some_html = """
<h1>Hello</h1>
<p>This is a paragraph</p>
<p class = "second">This is another paragraph</p>
<div>
<a href = 'www.google.com'><p>Your Friend</p></a>
"""

#### `BeautifulSoup`

To located elements in HTML, we use the `BeautifulSoup` library.  

In [ ]:
#make the soup


In [ ]:
#find first h1 tag


In [ ]:
# find first p tag


In [ ]:
# find all paragraphs


In [ ]:
#extract text from p tags


In [ ]:
#extract based on css 


### Getting HTML

Usually, we want to use a webpage to extract information from.  To get the HTML we use `requests` and turn the text of the response into `BeautifulSoup` objects.

In [ ]:
url = 'https://pitchfork.com/reviews/albums/'

In [ ]:
#make a request


In [ ]:
#turn it to soup


#### Finding elements

Typically, this is a bit of a dance with the inspect tool in your browswer.  Let's try to find each individual review as a start.

In [ ]:
# first review


In [ ]:
# first image


In [ ]:
# extract url


In [ ]:
from IPython.display import Image

In [ ]:
# visualize album cover


#### Extracting data from reviews

- Album
- Artist
- Genre
- Reviewer
- When
- Cover Art
- Full review url

### Getting more data

Looking at the url perhaps there is an idea for how to extract the first 10 pages of review data.

### Getting full review and score

Now, we have url's to our full reviews.  Let's use these to extract the score and full text of the review.

### Good vs. Bad

What score would you say makes an album good vs. a score that should be considered bad?

### Text Representation

To use this in a model we need to turn our text into a numeric representation.  A basic approach to this is to use the count of individual words as features.  Here, we can use the `CountVectorizer` to transform the data.

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>the</th>      <th>dog</th>      <th>ate</th>      <th>salami</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>0</td>      <td>1</td>      <td>1</td>      <td>1</td>    </tr>    <tr>      <th>1</th>      <td>1</td>      <td>1</td>      <td>0</td>      <td>1</td>    </tr>  </tbody></table>


In [ ]:
# instantiate count vectorizer


In [ ]:
# fit and transform the data


In [ ]:
# kind of thing?


In [ ]:
# convert to dense


In [ ]:
# words?


In [ ]:
# dataframe


In [ ]:
# pipeline


In [ ]:
# model


In [ ]:
# score it


In [ ]:
# coefficients


### `selenium`

Selenium allows you to interact with the webpage directly using a webdriver.  

https://selenium-python.readthedocs.io/installation.html#drivers

In [ ]:
# !pip install -U selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")

In [ ]:
#find element
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
driver.close()

#### Important Examples

In [ ]:
url = 'https://www.slapmagazine.com/'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

'''<input class="search_input" type="text" name="search" value="Search..." 
onfocus="this.value = '';" onblur="if(this.value=='') this.value='Search...';">'''

In [ ]:
element = driver.find_element(By.XPATH, "//input[@class='search_input']")

In [ ]:
element.send_keys("Drake")

In [ ]:
results = element.submit()

In [ ]:
elements = driver.find_elements(By.XPATH, "//div[@class='search_results_posts']")

In [ ]:
next_page = driver.find_element(By.XPATH, "//a[@class='navPages']")

In [ ]:
next_page.click()

In [ ]:
next_page = driver.find_elements(By.XPATH, "//a[@class='navPages']")

In [ ]:
elements = driver.find_elements(By.XPATH, "//div[@class='search_results_posts']")

In [ ]:
pages = driver.find_elements(By.XPATH, "//a[@class='navPages']")

In [ ]:
pages[1].click()

In [ ]:
pages = driver.find_elements(By.XPATH, "//a[@class='navPages']")

In [ ]:
pages[2].click()

In [ ]:
url = 'https://www.slapmagazine.com/'
driver = webdriver.Chrome()
driver.get(url)
element = driver.find_element(By.XPATH, "//input[@class='search_input']")
element.send_keys("Drake")
element.submit()


titles = []
for i in range(5):
    elements = driver.find_elements(By.XPATH, "//div[@class='search_results_posts']")
    for elem in elements:
        titles.append(elem.text)
    pages = driver.find_elements(By.XPATH, "//a[@class='navPages']")
    pages[i].click()

In [ ]:
driver.close()